In [ ]:
import torch
import dm_codes.datamatrix_provider
import dm_codes.datasets
import dm_codes.training
import lightning.pytorch as pl
import PIL.Image as Image


/home/pk/miniconda3/envs/dp-dm-conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import itertools
import torch.utils
import torch.utils.data


ds = dm_codes.datasets.DMIterableDataset(dm_codes.datamatrix_provider.DataMatrixProvider())
ds = list(itertools.islice(ds, 5000))
for x in ds:
    x["image"] = x.pop("corrupted")

dataloader = torch.utils.data.DataLoader(
    dataset=ds,
    batch_size=64
)


In [3]:
ds[0]

{'target': array([[[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]]], dtype=float32),
 'text': 'ip30OjZVn535T3t7iZCCdNxgXGEmS',
 'image': array([[[0.8745098 , 0.87058824, 0.8666667 , ..., 0.93333334,
          0.93333334, 0.9137255 ],
         [0.8666667 , 0.8666667 , 0.8666667 , ..., 0.93333334,
          0.93333334, 0.9137255 ],
         [0.8666667 , 0.8666667 , 0.8666667 , ..., 0.93333334,
          0.93333334, 0.9137255 ],
         ...,
         [0.8666667 , 0.8666667 , 0.8666667 , ..., 0.8666667 ,
          0.87058824, 0.87058824],
         [0.8666667 , 0.87058824, 0.87058824, ..., 0.8666667 ,
          0.8666667 , 0.87058824],
         [0.8666667 , 0.87058824, 0.87058824, ..., 0.8666667 ,
          0.8666667 , 0.8666667 ]],
 
        [[0.76862746, 0.7529412 , 0.7490196 , ..., 0.92156863,

In [ ]:
# load model from checkpoint and compute predictions
trainer = pl.Trainer(precision=16)
loaded_model = dm_codes.training.LitAutoEncoder.load_from_checkpoint(
    "../checkpoints/golden-bush-203/step=1000--corr_dec=0.5545.ckpt"
)
loaded_model.eval()
None

/home/pk/miniconda3/envs/dp-dm-conda/lib/python3.9/site-packages/lightning/fabric/connector.py:562: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/pk/miniconda3/envs/dp-dm-conda/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by def

In [5]:
predictions = trainer.predict(loaded_model, dataloader)
predictions = list(itertools.chain(*predictions))

You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/pk/miniconda3/envs/dp-dm-conda/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, predict_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting DataLoader 0: 100%|██████████| 79/79 [00:06<00:00, 12.84it/s]


In [ ]:
import pandas as pd
import numpy as np
import dm_codes.utils

pd_dataset = pd.DataFrame({
    "nn_prediction": predictions
})

In [7]:
pd_dataset.iloc[0][0].shape

/tmp/ipykernel_530697/4221189936.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pd_dataset.iloc[0][0].shape


torch.Size([1, 128, 128])

In [ ]:
np_prediction_for_image = pd_dataset["nn_prediction"].map(lambda x: np.squeeze(dm_codes.utils.tensor_to_numpy_for_image(x.unsqueeze(dim=0))))
pd_dataset["nn_output_image"] = np_prediction_for_image.map(lambda x: Image.fromarray(x, mode="L"))
pd_dataset["decoded_text_after_nn"] = np_prediction_for_image.map(dm_codes.utils.decode_dm_code)


Caught BrokenProcessPool exception while decoding image: BrokenProcessPool('A process in the process pool was terminated abruptly while the future was running or pending.')
dm_code_padded.min()=0, dm_code_padded.max()=255


In [9]:
pd_dataset["decoded_text_after_nn"]

0                               None
1                                  T
2                   Vfts6whx5oz64MOd
3                               None
4       qrRZbSs5HCJKCscIYHD9t2fgB97W
                    ...             
4995                             uWh
4996                   QBnfmk5wfXoy9
4997                   xnaEJVFuNnH6U
4998                            None
4999                            None
Name: decoded_text_after_nn, Length: 5000, dtype: object

In [10]:
pd_dataset["original_text"] = [x["text"] for x in ds]

In [11]:
decoded = pd_dataset[pd_dataset["decoded_text_after_nn"].notna()]
incorrectly_decoded = decoded[decoded["original_text"] != decoded["decoded_text_after_nn"]]
incorrectly_decoded = incorrectly_decoded[["original_text", "decoded_text_after_nn"]]
incorrectly_decoded

,original_text,decoded_text_after_nn
119,6R,P
215,os,o=02
305,9,72
449,CB6av,C6`s
721,FRj,ERj
767,YAx,AW
933,rWyE8,76
969,W7smqVYLsf0Fey9,28}
1123,gp,g`
1142,B,35


In [25]:
pd_dataset["original_text"].str.len().mean()

16.4748

In [18]:
decoded["original_text"].str.len().mean()

12.701209677419355

In [26]:
decoded["decoded_text_after_nn"].str.len().mean()

12.675

In [23]:
incorrectly_decoded["decoded_text_after_nn"].str.len().mean()

3.2916666666666665

In [12]:
import Levenshtein

In [13]:
incorrectly_decoded["lehven"] = [
    Levenshtein.distance(orig, decoded)
    for orig, decoded
    in zip(
        incorrectly_decoded["original_text"],
        incorrectly_decoded["decoded_text_after_nn"]
    )
]


In [14]:
incorrectly_decoded

,original_text,decoded_text_after_nn,lehven
119,6R,P,2
215,os,o=02,3
305,9,72,2
449,CB6av,C6`s,3
721,FRj,ERj,1
767,YAx,AW,2
933,rWyE8,76,5
969,W7smqVYLsf0Fey9,28},15
1123,gp,g`,1
1142,B,35,2


In [37]:
long = incorrectly_decoded[incorrectly_decoded["original_text"].str.len() >= 5]
long

,original_text,decoded_text_after_nn,lehven
449,CB6av,C6`s,3
933,rWyE8,76,5
969,W7smqVYLsf0Fey9,28},15
2026,W2r1a7EmwzFa84ELQkjLuutuHyyg1Rn,0172,29
3113,6vnYXX249,97`I16,9
3590,jfm9hk4XKqWtX4r17DkaSOu,|,23
3725,UP5Il9sWbgPJXzVXSsUeaF5RQk,H,26
4842,2zoFNMe9Juc,91k40,11


In [39]:
short = incorrectly_decoded[incorrectly_decoded["original_text"].str.len() < 5]
(short["lehven"] / short["original_text"].str.len()).value_counts().sort_index()

0.333333     3
0.500000     6
0.666667     5
1.000000    19
1.250000     1
1.500000     1
2.000000     2
3.000000     1
4.000000     2
Name: count, dtype: int64

In [38]:
len(long), len(short)

(8, 40)

In [43]:
(incorrectly_decoded["lehven"] / incorrectly_decoded["original_text"].str.len()).value_counts().sort_index()

0.333333     3
0.500000     6
0.600000     1
0.666667     5
0.935484     1
1.000000    25
1.250000     1
1.500000     1
2.000000     2
3.000000     1
4.000000     2
Name: count, dtype: int64

from 5000 generated DM codes, 48 were misread, encoded str len uniform 1-32

most misreads, 40/48 were very short strings (<5 chars)
=> long strings likely have better error correction

edit distance is almost always at least 0.5*len(encoded string),
only in 3 cases were smaller (always it was very short string)

overall:
**misread strings are not close to encoded strings**